<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/AGENTIC-MODULE6-FaultPredictionAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:


# ============================================================
# Fault Classification Pipeline
# ============================================================

############PASTE ADAPTIVE WINDOW HERE - so everything is in one file - later, we can import as a package#####################


# ====== AdaptiveWindowAgent ======
# =====================================================
# AdaptiveWindowAgent (improved version)
# =====================================================
import numpy as np
import pandas as pd
import pickle, os, logging, datetime as dt
from collections import deque
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.vector_ar.var_model import VAR
import keras

logger = logging.getLogger(__name__)

class AdaptiveWindowAgent:
    """
    Adaptive Window Agent:
    - Predicts window size using MLP
    - Evaluates forecast with VAR
    - Monitors anomalies & drift with adaptive thresholds
    - Outputs severity scores + suppresses redundant events
    """

    def __init__(self, agent_id="adaptive_window_agent",
                 model_path=None, checkpoint_path=None):
        self.agent_id = agent_id
        self.model_path = model_path or "/content/drive/MyDrive/PHD/2025/DGRNet-MLP-Versions/METROPM_MLP_model_Daily.keras"
        self.checkpoint_path = checkpoint_path

        # Core model
        self.model = None
        self.transformer = StandardScaler()
        self.transformer_fitted = False
        self.is_model_loaded = False

        # Histories
        self.prediction_history = deque(maxlen=1000)
        self.mse_history = deque(maxlen=200)
        self.mae_history = deque(maxlen=200)

        # Event detection params
        self.drift_detection_window = 20
        self.drift_threshold_mse = 1.5   # stricter
        self.drift_threshold_mae = 1.5
        self.consecutive_poor_predictions = 0
        self.cooldown_counter = 0

        # Stats
        self.performance_stats = {
            'total_predictions': 0,
            'avg_mse': 0.0,
            'avg_mae': 0.0,
            'last_retrain_time': None,
            'drift_events': 0,
            'anomaly_events': 0,
            'retraining_events': 0
        }

        # Retraining buffers
        self.retraining_data = {
            'x_buffer': deque(maxlen=10000),
            'y_buffer': deque(maxlen=10000)
        }

        self.load_model()
        print(f"AdaptiveWindowAgent {self.agent_id} initialized")

    # ------------------- Model -------------------

    def load_model(self):
        try:
            if os.path.exists(self.model_path):
                self.model = keras.models.load_model(self.model_path)
                self.is_model_loaded = True
                print(f"✅ Loaded MLP model from {self.model_path}")

                # Try to load transformer
                transformer_path = self.model_path.replace('.keras', '_transformer.pkl')
                if os.path.exists(transformer_path):
                    with open(transformer_path, 'rb') as f:
                        self.transformer = pickle.load(f)
                    self.transformer_fitted = True
                else:
                    # Fit transformer from true window labels
                    y_original = np.load(
                        "/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/generated-data-true-window2.npy"
                    )
                    self.transformer.fit(y_original.reshape(-1, 1))
                    self.transformer_fitted = True
                    with open(transformer_path, 'wb') as f:
                        pickle.dump(self.transformer, f)
                    print("⚠️ No transformer found, fitted a new one.")
            else:
                print(f"❌ Model not found at {self.model_path}")
        except Exception as e:
            print(f"❌ Error loading model: {e}")

    # ------------------- Forecast Eval -------------------
    def evaluate_forecast_performance(self, sequence_3d, predicted_window, n_future=1):
        try:
            df = pd.DataFrame(sequence_3d, columns=[f'V{i+1}' for i in range(sequence_3d.shape[1])])
            df_train, df_test = df[:-n_future], df[-n_future:]

            # Drop constant cols
            constant_cols = [c for c in df_train.columns if df_train[c].nunique() <= 1]
            df_train = df_train.drop(columns=constant_cols, errors="ignore")
            df_test = df_test.drop(columns=constant_cols, errors="ignore")

            # If too few variables, fall back immediately
            if len(df_train.columns) < 1:
                return self._persistence_forecast(df, df_test)

            k = min(predicted_window, len(df_train) - 2)
            if k < 1: k = 1

            # Try VAR
            try:
                model = VAR(df_train)
                model_fitted = model.fit(maxlags=k, trend="c")
                forecast_input = df_train.values[-model_fitted.k_ar:]
                fc = model_fitted.forecast(y=forecast_input, steps=n_future)
                df_forecast = pd.DataFrame(fc, index=df.index[-n_future:], columns=df_train.columns)

                actual = df_test[df_forecast.columns].values.flatten()
                predicted = df_forecast.values.flatten()

            except Exception:
                # Try AutoReg
                try:
                    from statsmodels.tsa.ar_model import AutoReg
                    col = df_train.columns[0]
                    model = AutoReg(df_train[col], lags=min(k, len(df_train)//2)).fit()
                    predicted = model.predict(start=len(df_train), end=len(df_train)+n_future-1).values
                    actual = df_test[col].values
                except Exception:
                    # Fallback to persistence
                    return self._persistence_forecast(df, df_test)

            mse = np.mean((actual - predicted) ** 2)
            mae = np.mean(np.abs(actual - predicted))

            # If forecast is unstable, fallback
            if np.isnan(mse) or np.isnan(mae) or mse > 10 or mae > 10:
                return self._persistence_forecast(df, df_test)

            return {
                'mse': float(mse),
                'mae': float(mae),
                'forecast_success': True,
                'actual_values': actual.tolist(),
                'predicted_values': predicted.tolist(),
                'used_columns': list(df_test.columns)
            }

        except Exception:
            return self._persistence_forecast(df, df_test)

# ------------------- Persistence fallback -------------------

    def _persistence_forecast(self, df, df_test):
        """Simple last-value-carried-forward forecast."""
        try:
            last_values = df.iloc[-1].values
            predicted = np.tile(last_values, (len(df_test), 1))
            actual = df_test.values.flatten()

            mse = np.mean((actual - predicted.flatten()) ** 2)
            mae = np.mean(np.abs(actual - predicted.flatten()))

            return {
                'mse': float(mse),
                'mae': float(mae),
                'forecast_success': True,
                'actual_values': actual.tolist(),
                'predicted_values': predicted.flatten().tolist(),
                'used_columns': list(df_test.columns),
                'note': 'persistence_fallback'
            }
        except Exception as e:
            return {'mse': 9999, 'mae': 9999, 'forecast_success': True, 'error': str(e), 'note': 'persistence_fallback_failed'}



    # ------------------- Prediction -------------------

    def predict_window_size(self, feature_vector, sequence_3d):
        if not self.is_model_loaded:
            return {'predicted_window': 20, 'error': "Model not loaded"}

        try:
            if feature_vector.ndim == 1:
                feature_vector = feature_vector.reshape(1, -1)

            pred_raw = self.model.predict(feature_vector, verbose=0)
            if self.transformer_fitted:
                predicted_window = int(round(self.transformer.inverse_transform(pred_raw)[0, 0]))
            else:
                predicted_window = int(round(pred_raw[0, 0]))

            # Evaluate
            forecast_metrics = self.evaluate_forecast_performance(sequence_3d, predicted_window, n_future=1)

            if forecast_metrics.get("forecast_success", False):
                self.mse_history.append(forecast_metrics["mse"])
                self.mae_history.append(forecast_metrics["mae"])
                self.performance_stats['total_predictions'] += 1
                self.performance_stats['avg_mse'] = np.mean(self.mse_history)
                self.performance_stats['avg_mae'] = np.mean(self.mae_history)

            # Event check
            event, sev = self._check_for_event()

            # Save history
            record = {
                'timestamp': dt.datetime.now(),
                'predicted_window': predicted_window,
                'forecast_metrics': forecast_metrics,
                'event_type': event,
                'severity': sev
            }
            self.prediction_history.append(record)

            return {
                'predicted_window': predicted_window,
                'forecast_metrics': forecast_metrics,
                'event_type': event,
                'severity': sev,
                'performance_stats': self.get_recent_performance()
            }
        except Exception as e:
            return {'predicted_window': 20, 'error': str(e)}

    # ------------------- Event Logic -------------------

    def _check_for_event(self):
        if len(self.mse_history) < self.drift_detection_window:
            return None, 0.0

        mse_vals = np.array(self.mse_history)[-self.drift_detection_window:]
        mae_vals = np.array(self.mae_history)[-self.drift_detection_window:]

        # Rolling stats
        mean_mse, std_mse = np.mean(mse_vals), np.std(mse_vals) + 1e-8
        last_mse = mse_vals[-1]

        # Normalized error
        norm_error = (last_mse - mean_mse) / std_mse

        # Severity
        anomaly_severity = max(0, norm_error)
        drift_severity = max(0, (np.mean(mse_vals) / (np.median(mse_vals)+1e-5)) - 1)

        # Check anomaly
        if last_mse > mean_mse + 2.0 * std_mse:
            self.performance_stats['anomaly_events'] += 1
            return "ANOMALY", anomaly_severity

        # Check drift (with persistence + cooldown)
        ema_mse = 0.3*np.mean(mse_vals) + 0.7*np.median(mse_vals)
        ema_mae = 0.3*np.mean(mae_vals) + 0.7*np.median(mae_vals)
        mse_ratio = ema_mse / max(np.median(mse_vals), 1e-5)
        mae_ratio = ema_mae / max(np.median(mae_vals), 1e-5)

        if mse_ratio > self.drift_threshold_mse and mae_ratio > self.drift_threshold_mae:
            self.consecutive_poor_predictions += 1
            if self.consecutive_poor_predictions >= 5 and self.cooldown_counter == 0:
                self.performance_stats['drift_events'] += 1
                self.cooldown_counter = 10
                return "DRIFT", drift_severity
        else:
            self.consecutive_poor_predictions = 0

        if self.cooldown_counter > 0:
            self.cooldown_counter -= 1

        return None, 0.0

    # ------------------- Helpers -------------------

    def get_recent_performance(self):
        successful_predictions = [
            p for p in list(self.prediction_history)[-50:]
            if p.get('forecast_metrics', {}).get('forecast_success', False)
        ]
        return {
            'total_predictions': len(self.prediction_history),
            'successful_predictions': len(successful_predictions),
            'success_rate': len(successful_predictions) / max(len(self.prediction_history), 1),
            'drift_events': self.performance_stats['drift_events'],
            'anomaly_events': self.performance_stats['anomaly_events'],
            'retraining_events': self.performance_stats['retraining_events'],
            'recent_mse': float(np.mean(list(self.mse_history)[-10:])) if self.mse_history else 0,
            'avg_mse': float(np.mean(self.mse_history)) if self.mse_history else 0,
            'recent_mae': float(np.mean(list(self.mae_history)[-10:])) if self.mae_history else 0,
            'avg_mae': float(np.mean(self.mae_history)) if self.mae_history else 0,
            'transformer_fitted': self.transformer_fitted
        }


    def save_performance_state(self, filepath: str):
        """Save performance statistics + prediction history to JSON"""
        try:
            state = {
                'performance_stats': self.performance_stats.copy(),
                'prediction_history': list(self.prediction_history)[-100:],  # last 100
                'mse_history': list(self.mse_history),
                'mae_history': list(self.mae_history),
                'transformer_fitted': self.transformer_fitted
            }
            import json
            with open(filepath, 'w') as f:
                json.dump(state, f, indent=2, default=str)
            print(f"✅ Performance state saved to {filepath}")
        except Exception as e:
            print(f"❌ Failed to save performance state: {e}")


#############################################
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import joblib

# ============================================================
# 1. Load Data
# ============================================================
# Long subsequences (length=50) and labels
long_sequences = np.load("/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/multivariate_long_sequences-TRAIN-Daily-DIRECT-VAR.npy")
labels_detection = np.load("/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/anomaly_labels_detection.npy")  # fault labels
labels_h1 = np.load("/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/anomaly_labels_H1.npy")  # optional predictive labels

print("Data loaded:", long_sequences.shape, labels_detection.shape)

# ============================================================
# 2. Baseline Fixed Window Features (window=50, flattened)
# ============================================================
X_fixed = long_sequences.reshape(long_sequences.shape[0], -1)
y = labels_detection  # detection labels (can switch to h1/h3/h12)

X_train, X_test, y_train, y_test = train_test_split(X_fixed, y, test_size=0.2, random_state=42, stratify=y)

print("Train/Test shapes:", X_train.shape, X_test.shape)

# Baseline-1: Random Forest
rf = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("\n=== Baseline-1: Fixed Window Random Forest ===")
print(classification_report(y_test, y_pred_rf, digits=4))
print(confusion_matrix(y_test, y_pred_rf))
joblib.dump(rf, "rf_fixed_window.pkl")

# ============================================================
# 3. Dynamic Window Features (using your AdaptiveWindowAgent)
# ============================================================
#from agents.adaptive_window_agent import AdaptiveWindowAgent

window_agent = AdaptiveWindowAgent(
    model_path="/content/drive/MyDrive/PHD/2025/DGRNet-MLP-Versions/METROPM_MLP_model_Daily.keras"
)

# Generate features for each subsequence using predicted window
dynamic_features = []
for seq in long_sequences:
    features = seq.flatten()
    result = window_agent.predict_window_size(features, seq)
    w = result.get("predicted_window", 50)
    # Extract the last w timesteps from the long sequence
    seq_dynamic = seq[-w:].flatten()
    dynamic_features.append(seq_dynamic)

# Pad to same length (use max window=50)
X_dynamic = np.array([np.pad(f, (0, 50*seq.shape[1] - len(f))) for f in dynamic_features])

X_train_dyn, X_test_dyn, y_train_dyn, y_test_dyn = train_test_split(X_dynamic, y, test_size=0.2, random_state=42, stratify=y)

# Baseline-12: Random Forest + dynamic window
rf = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train_dyn, y_train_dyn)
y_pred_rf_dyn = rf.predict(X_test_dyn)

print("\n=== Baseline-1: Fixed Window Random Forest ===")
print(classification_report(y_test_dyn, y_pred_rf_dyn, digits=4))
print(confusion_matrix(y_test_dyn, y_pred_rf_dyn))
joblib.dump(rf, "rf_dynamic_window.pkl")

# Baseline-3: XGBoost fixed
xgb = XGBClassifier(
    n_estimators=300, max_depth=8, learning_rate=0.05,
    subsample=0.8, colsample_bytree=0.8,
    random_state=42, n_jobs=-1
)
xgb.fit(X_train_dyn, y_train_dyn)
y_pred_xgb = xgb.predict(X_test_dyn)

print("\n=== Baseline-2: Dynamic Window XGBoost ===")
print(classification_report(y_test_dyn, y_pred_xgb, digits=4))
print(confusion_matrix(y_test_dyn, y_pred_xgb))
joblib.dump(xgb, "xgb_dynamic_window.pkl")


# ============================================================
# Ready for Next Steps
# ============================================================
print("\n✅ Baseline models trained and evaluated.")
print("Next: add Transformer-based sequence classifier, then coordinator features.")



# ========================================================================================
# SOTA: Transformer (PatchTST-style) Classifier
# =======================================================================================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.metrics import Precision, Recall
import tensorflow.keras.backend as K
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# ====================
# 1. Mixed Precision
# ====================
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

# ====================
# 2. Define Transformer Block
# ====================
def transformer_block(inputs, head_size=32, num_heads=2, ff_dim=64, dropout=0.2):
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    ff = layers.Dense(ff_dim, activation="relu")(x)
    ff = layers.Dropout(dropout)(ff)
    ff = layers.Dense(inputs.shape[-1])(ff)
    out = layers.LayerNormalization(epsilon=1e-6)(x + ff)
    return out

# ====================
# 3. Build Model
# ====================
def build_transformer_classifier(input_shape, num_classes=1):
    inputs = keras.Input(shape=input_shape)

    # Smaller Transformer (2 layers)
    x = transformer_block(inputs, head_size=32, num_heads=2, ff_dim=64, dropout=0.2)
    x = transformer_block(x, head_size=32, num_heads=2, ff_dim=64, dropout=0.2)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    if num_classes == 1:
        outputs = layers.Dense(1, activation="sigmoid", dtype="float32")(x)
    else:
        outputs = layers.Dense(num_classes, activation="softmax", dtype="float32")(x)

    return keras.Model(inputs, outputs)

# ====================
# 4. Data Setup
# ====================
long_sequences = np.load("/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/multivariate_long_sequences-TRAIN-Daily-DIRECT-VAR.npy")
labels_detection = np.load("/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/anomaly_labels_detection.npy")

X = long_sequences
y = labels_detection

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train shape:", X_train.shape, y_train.shape)
print("Val shape:", X_val.shape, y_val.shape)

# Class weights
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weight_dict = {i: w for i, w in zip(classes, class_weights)}
print("Class weights:", class_weight_dict)

# ====================
# 5. Compile + Train
# ====================

# Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        bce = K.binary_crossentropy(y_true, y_pred)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_factor = y_true * alpha + (1 - y_true) * (1 - alpha)
        modulating = (1 - p_t) ** gamma
        return K.mean(alpha_factor * modulating * bce, axis=-1)
    return loss

model = build_transformer_classifier(input_shape=X_train.shape[1:], num_classes=1)

# ✅ Compile properly
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=focal_loss(gamma=2, alpha=0.25),
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall")]
)

# Callbacks
early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
lr_sched = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6)

# Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=128,
    class_weight=class_weight_dict,
    callbacks=[early_stop, lr_sched],
    verbose=1
)

# ====================
# 6. Evaluate
# ====================
y_probs = model.predict(X_val).flatten()
y_pred = (y_probs >= 0.5).astype(int)

print("\n=== Transformer Evaluation ===")
print(classification_report(y_val, y_pred, digits=4))
print(confusion_matrix(y_val, y_pred))


#######################################################################################################
#Validate with coordination agent###############################
#######################################################################################################
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

def validate_with_coordinator(y_probs, y_true, coordinator_results, threshold=0.5):
    """
    Validate Transformer predictions against coordinator agent decisions.
    """
    # Transformer raw predictions
    y_pred_raw = (y_probs >= threshold).astype(int)

    # Apply coordinator filter
    y_pred_validated = []
    for pred, coord in zip(y_pred_raw, coordinator_results):
        coord_flag = coord["coordinator"].get("final_anomaly", False)
        if pred == 1 and coord_flag:
            y_pred_validated.append(1)   # confirmed
        else:
            y_pred_validated.append(0)   # suppress FP or keep 0

    y_pred_validated = np.array(y_pred_validated)

    # Reports
    report_raw = classification_report(y_true, y_pred_raw, output_dict=True)
    report_validated = classification_report(y_true, y_pred_validated, output_dict=True)

    cm_raw = confusion_matrix(y_true, y_pred_raw)
    cm_validated = confusion_matrix(y_true, y_pred_validated)

    return {
        "report_raw": report_raw,
        "report_validated": report_validated,
        "confusion_matrix_raw": cm_raw,
        "confusion_matrix_validated": cm_validated,
        "raw_preds": y_pred_raw,
        "validated_preds": y_pred_validated
    }

import matplotlib.pyplot as plt
import seaborn as sns

def plot_confusion_matrices(cm_raw, cm_validated, labels=[0,1]):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    sns.heatmap(cm_raw, annot=True, fmt="d", cmap="Blues",
                xticklabels=labels, yticklabels=labels, ax=axes[0])
    axes[0].set_title("Transformer (Raw)")
    axes[0].set_xlabel("Predicted")
    axes[0].set_ylabel("True")

    sns.heatmap(cm_validated, annot=True, fmt="d", cmap="Greens",
                xticklabels=labels, yticklabels=labels, ax=axes[1])
    axes[1].set_title("Transformer + Coordinator")
    axes[1].set_xlabel("Predicted")
    axes[1].set_ylabel("True")

    plt.tight_layout()
    plt.show()
# ==========================================
# Generate Coordinator outputs for test set
# ==========================================


results = []
coordinator_flags = []

print("Running Coordinator over test set...")

for i, seq in enumerate(X_test_long):
    try:
        features = seq.flatten()

        # run each agent
        master_out = master.process_system_input(seq)
        window_out = window_agent.predict_window_size(features, seq)
        final = coordinator.fuse(master_out, window_out)

        results.append({
            "master": master_out,
            "window": window_out,
            "coordinator": final
        })

        # Store just the binary anomaly flag (1=anomaly, 0=normal)
        coordinator_flags.append(1 if final["final_anomaly"] else 0)

    except Exception as e:
        print(f"Sample {i} failed: {e}")
        coordinator_flags.append(0)  # safe default

coordinator_flags = np.array(coordinator_flags)
print(f"✅ Coordinator outputs generated: {coordinator_flags.shape}")
print(f"Unique flags: {np.unique(coordinator_flags, return_counts=True)}")



validation_out = validate_with_coordinator(
    y_probs=y_probs,
    y_true=y_true,
    coordinator_results=results,
    threshold=0.5
)

print("=== Transformer (Raw) ===")
print(validation_out["report_raw"])
print(validation_out["confusion_matrix_raw"])

print("\n=== Transformer + Coordinator (Validated) ===")
print(validation_out["report_validated"])
print(validation_out["confusion_matrix_validated"])

plot_confusion_matrices(
    validation_out["confusion_matrix_raw"],
    validation_out["confusion_matrix_validated"]
)

Train shape: (2901, 50, 12) (2901,)
Val shape: (726, 50, 12) (726,)
Class weights: {np.int64(0): np.float64(0.5125441696113074), np.int64(1): np.float64(20.429577464788732)}
Epoch 1/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 31s 669ms/step - accuracy: 0.8704 - loss: 0.0622 - precision: 0.0794 - recall: 0.4089 - val_accuracy: 0.9325 - val_loss: 0.0408 - val_precision: 0.1702 - val_recall: 0.4444 - learning_rate: 0.0010
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 17s 10ms/step - accuracy: 0.9442 - loss: 0.0451 - precision: 0.2169 - recall: 0.5220 - val_accuracy: 0.9284 - val_loss: 0.0339 - val_precision: 0.1852 - val_recall: 0.5556 - learning_rate: 0.0010
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9403 - loss: 0.0428 - precision: 0.2579 - recall: 0.6432 - val_accuracy: 0.9063 - val_loss: 0.0334 - val_precision: 0.1711 - val_recall: 0.7222 - learning_rate: 0.0010
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9348 - loss: 0.0331 - precision: 0.2942 - recall: 0.8725 - 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')